In [1]:
import os
import numpy as np
import pandas as pd
from random import randint,seed
from tqdm import tqdm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import PolynomialFeatures

In [2]:
"""
df1 = pd.read_csv('data_conv/new_dataset/conv1.csv')
df1['freq'] = 1200
print (df1.shape)
df2 = pd.read_csv('data_conv/new_dataset/conv2.csv')
df2['freq'] = 800
print (df2.shape)
df3 = pd.read_csv('data_conv/new_dataset/conv3.csv')
df3['freq'] = 600
print (df3.shape)
df4 = pd.read_csv('data_conv/new_dataset/conv4.csv')
df4['freq'] = 1000
print (df4.shape)
"""
df1 = pd.read_csv('data_conv/conv1.csv')
df1['freq'] = 1200
print (df1.shape)
df2 = pd.read_csv('data_conv/conv2.csv')
df2['freq'] = 800
print (df2.shape)
df3 = pd.read_csv('data_conv/conv3.csv')
df3['freq'] = 600
print (df3.shape)
df4 = pd.read_csv('data_conv/conv4.csv')
df4['freq'] = 1000
print (df4.shape)

(1346000, 9)
(708000, 9)
(461400, 9)
(1052000, 9)


In [3]:
df = pd.concat([df1, df2,df3,df4], ignore_index=True)

In [4]:
df.head()

,Unnamed: 0,input_size_h,input_size_w,channels,filter_count,filter_size,stride,runtime,freq
0,0,562,20,900,85,4,2,3.582578,1200
1,1,33,53,8,3,2,1,0.002315,1200
2,2,33,53,13,20,10,1,0.075039,1200
3,3,13,33,5,85,8,1,0.008288,1200
4,4,13,53,5,85,10,1,0.014262,1200


In [5]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,input_size_h,input_size_w,channels,filter_count,filter_size,stride,runtime,freq
0,562,20,900,85,4,2,3.582578,1200
1,33,53,8,3,2,1,0.002315,1200
2,33,53,13,20,10,1,0.075039,1200
3,13,33,5,85,8,1,0.008288,1200
4,13,53,5,85,10,1,0.014262,1200


In [6]:
df_ = df.dropna(axis=0)
df_.head()

,input_size_h,input_size_w,channels,filter_count,filter_size,stride,runtime,freq
0,562,20,900,85,4,2,3.582578,1200
1,33,53,8,3,2,1,0.002315,1200
2,33,53,13,20,10,1,0.075039,1200
3,13,33,5,85,8,1,0.008288,1200
4,13,53,5,85,10,1,0.014262,1200


In [7]:
df_['runtime_log'] = np.log(df_.runtime)
#df_['runtime_log'] = df_.runtime

/home/samavatian.1/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
/home/samavatian.1/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
df_ = df_.dropna(axis=0)

In [9]:
def linear_reg_poly(df_,D):
    target = df_.runtime_log
    data = df_[['freq','input_size_h','input_size_w' ,'channels','filter_count' ,'filter_size' ,'stride']]
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=0)
    #np.set_printoptions(precision=10,suppress=True)
    #print np.asarray(X_train.mean())
    #print np.asarray(X_train.std())
    scaler = StandardScaler() 
    scaler.fit(X_train)  
    X_train_scale = scaler.transform(X_train)
    
    poly = PolynomialFeatures(degree=D,include_bias=False).fit(X_train_scale)
    
    X_train_poly = poly.transform(X_train_scale)
    
    #print("X_train.shape: {}".format(X_train_scale.shape))
    #print("X_train_poly.shape: {}".format(X_train_poly.shape))
    #print("Polynomial feature names:\n{}".format(poly.get_feature_names()))
    
    X_test_scale = scaler.transform(X_test)
    X_test_poly = poly.transform(X_test_scale)
    alp = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    temp=[]
    for a in alp:
        
        ret=[]
        ret.append(D)
        model = linear_model.Lasso(alpha=a)
        model.fit(X_train_poly, y_train)
        #print('accuracy: ',model.score(X_train_poly, y_train)*100)
        print 'aplph: ',a
        ret.append(a)
        acc = model.score(X_test_poly, y_test)*100
        print 'accuracy: ',acc
        ret.append(acc)
        print 'features: ',model.coef_.shape[0]
        ret.append(model.coef_.shape[0])
        print 'nonzero features: ', np.count_nonzero(model.coef_)
        ret.append(np.count_nonzero(model.coef_))
        temp.append(ret)
    return temp
    #print model.sparse_coef_
    #print model.intercept_
    

In [36]:
def aug_features(df_):
    df_['met1'] = df_.freq**2
    df_['met2'] = df_.input_size_h**2
    df_['met3'] = df_.input_size_w**2
    df_['met4'] = df_.channels**2
    df_['met5'] = df_.filter_count**2
    df_['met6'] = df_.filter_size**2
    df_['met7'] = df_.stride**2
    
    df_['met8'] = df_.freq**3
    df_['met9'] = df_.input_size_h**3
    df_['met10'] = df_.input_size_w**3
    df_['met11'] = df_.channels**3
    df_['met12'] = df_.filter_count**3
    df_['met13'] = df_.filter_size**3
    df_['met14'] = df_.stride**3
    
    df_['met15'] = df_.freq*df_.channels
    df_['met16'] = df_.input_size_h*df_.channels
    df_['met17'] = df_.input_size_w*df_.channels
    df_['met18'] = df_.channels**4
    df_['met19'] = df_.filter_count*df_.channels
    df_['met20'] = df_.filter_size*df_.channels
    df_['met21'] = df_.stride*df_.channels
    
    df_['met22'] = df_.freq*df_.channels**2
    df_['met23'] = df_.input_size_h*df_.channels**2
    df_['met24'] = df_.input_size_w*df_.channels**2
    df_['met25'] = df_.channels**5
    df_['met26'] = df_.filter_count*df_.channels**2
    df_['met27'] = df_.filter_size*df_.channels**2
    df_['met28'] = df_.stride*df_.channels**2
    
    """
    df_['met29'] = df_.input_size_h*df_.channels**3
    df_['met30'] = df_.input_size_w*df_.channels**3
    df_['met31'] = df_.channels**6
    df_['met32'] = df_.filter_count*df_.channels**3
    df_['met33'] = df_.filter_size*df_.channels**3
    df_['met34'] = df_.stride*df_.channels**3
    df_['met35'] = df_.freq*df_.channels**3
    """
    return df_

def linear_reg(df_):
    target = df_.runtime_log
    
    data =  df_[['freq','input_size_h','input_size_w' ,'channels','filter_count' ,'filter_size' ,'stride']]
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=0)
    mean = np.asarray(X_train.mean())
    std = np.asarray(X_train.std())
    print mean
    print std
    
    scaler = StandardScaler() 
    scaler.fit(X_train)  
    """
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    """
    X_train = (X_train-mean)/std
    X_test =(X_test-mean)/std
    X_train = pd.DataFrame(X_train,columns=['freq','input_size_h','input_size_w' ,'channels','filter_count' ,'filter_size' ,'stride'])
    X_test = pd.DataFrame(X_test,columns=['freq','input_size_h','input_size_w' ,'channels','filter_count' ,'filter_size' ,'stride'])
    X_train = aug_features(X_train)
    X_test = aug_features(X_test)

    model = LinearRegression()
    model.fit(X_train, y_train)
    
    print('accuracy: ',model.score(X_train, y_train)*100)
    print('accuracy: ',model.score(X_test, y_test)*100)
    print (model.predict(X_test))
    print model.coef_
    print model.intercept_
    return scaler,model,mean,std
    
    

In [37]:
scaler,model,mean,std = linear_reg(df_)

[983.96255419 141.9505058  142.06561549 267.2092365  273.72016019
   5.37187517   1.50360116]
[208.53699603 170.77112745 170.70346169 403.1205735  412.33922581
   2.81409736   0.49998712]
('accuracy: ', 97.4158231929509)
('accuracy: ', 97.42666908909557)
[-0.92428746 -1.12432247 -4.20174654 ... -4.94006007 -0.95462785
 -1.70145453]
[-1.94703716e-01  1.79868392e+00  1.79885749e+00  2.51867462e-02
  1.83357685e+00  1.06345284e+00 -2.73457611e+08  2.64709189e-02
 -1.71988000e+00 -1.72318608e+00 -9.32783736e-01 -1.23576421e+00
 -3.69442574e-01 -2.18576768e+07 -4.27025184e-03  4.82196368e-01
  4.83110430e-01  6.67630266e+00  2.73069821e-01  8.38139682e-02
  2.73086181e+08  5.51603540e-03  6.40674153e-03  1.24048655e-03
 -5.46737480e+00 -4.15892806e-02  1.58341405e-01  2.44728852e-04
 -4.67562498e-03 -3.87315619e-03 -1.58086196e-03  1.13554722e+00
  1.43174281e-02 -5.23012225e-02  6.60040677e-04]
25791475.435509738


In [38]:
l = [[1000,256,256,3,64,3,1],[1000,32,32,512,512,3,1],[1000,14,14,512,1024,3,1],[1000,27,27,96,256,5,1]]
l = np.asarray(l)
l = scaler.transform(l)
ldf = pd.DataFrame(l,columns=['freq','input_size_h','input_size_w' ,'channels','filter_count' ,'filter_size' ,'stride'])
l = aug_features(ldf)
l


,freq,input_size_h,input_size_w,channels,filter_count,filter_size,stride,met1,met2,met3,...,met19,met20,met21,met22,met23,met24,met25,met26,met27,met28
0,0.076905,0.667850,0.667441,-0.65541,-0.508611,-0.842855,-1.007228,0.005914,0.446024,0.445477,...,0.333349,0.552416,0.660148,0.033035,0.286883,0.286707,-0.120939,-0.218480,-0.362059,-0.432667
1,0.076905,-0.643847,-0.644777,0.60724,0.577873,-0.842855,-1.007228,0.005914,0.414539,0.415737,...,0.350908,-0.511815,-0.611629,0.028358,-0.237412,-0.237755,0.082566,0.213085,-0.310794,-0.371405
2,0.076905,-0.749252,-0.750223,0.60724,1.819570,-0.842855,-1.007228,0.005914,0.561378,0.562834,...,1.104915,-0.511815,-0.611629,0.028358,-0.276279,-0.276637,0.082566,0.670948,-0.310794,-0.371405
3,0.076905,-0.673126,-0.674067,-0.42471,-0.042975,-0.132147,-1.007228,0.005914,0.453099,0.454367,...,0.018252,0.056124,0.427780,0.013872,-0.121417,-0.121587,-0.013819,-0.007752,-0.023837,-0.181682


In [39]:
np.exp(model.predict(l))

array([1.93084893e-47, 3.60833445e-47, 1.51379897e-47, 9.79032011e-48])

In [40]:
m = [[1000,256,256,3,64,3,1],[1000,32,32,512,512,3,1],[1000,14,14,512,1024,3,1],[1000,27,27,96,256,5,1]]
m = np.asarray(m)
m_ = (m - mean)/std
mdf = pd.DataFrame(m_,columns=['freq','input_size_h','input_size_w' ,'channels','filter_count' ,'filter_size' ,'stride'])
m_ = aug_features(mdf)
m_

,freq,input_size_h,input_size_w,channels,filter_count,filter_size,stride,met1,met2,met3,...,met19,met20,met21,met22,met23,met24,met25,met26,met27,met28
0,0.076905,0.667850,0.667440,-0.65541,-0.508611,-0.842855,-1.007228,0.005914,0.446024,0.445477,...,0.333349,0.552415,0.660147,0.033035,0.286883,0.286707,-0.120939,-0.218480,-0.362059,-0.432667
1,0.076905,-0.643847,-0.644777,0.60724,0.577873,-0.842855,-1.007228,0.005914,0.414539,0.415737,...,0.350908,-0.511815,-0.611629,0.028358,-0.237412,-0.237755,0.082566,0.213085,-0.310794,-0.371405
2,0.076905,-0.749251,-0.750223,0.60724,1.819569,-0.842855,-1.007228,0.005914,0.561378,0.562834,...,1.104915,-0.511815,-0.611629,0.028358,-0.276279,-0.276637,0.082566,0.670948,-0.310794,-0.371405
3,0.076905,-0.673126,-0.674067,-0.42471,-0.042975,-0.132147,-1.007228,0.005914,0.453099,0.454367,...,0.018252,0.056124,0.427780,0.013872,-0.121417,-0.121587,-0.013819,-0.007752,-0.023837,-0.181682


In [41]:
np.exp(model.predict(m_))

array([0.49340082, 0.92205698, 0.38682931, 0.25017743])

In [42]:
diff = ldf-mdf